In [1]:
import pymysql
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [5]:
dict0 = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\脾脏\dict.xlsx")
id_list = dict0.loc[:,'TagID'].values.tolist()
id_list

['OBDIM0005EX',
 'OBDIM0005Tag0001',
 'OBDIM0005Tag0002',
 'OBDIM0005Tag0003',
 'OBDIM0005Tag0004',
 'OBDIM0005Tag0005',
 'OBDIM0005Tag0006',
 'OBDIM0005Tag0007',
 'OBDIM0005Tag0008',
 'OBDIM0005Tag0009',
 'OBDIM0005Tag0010',
 'OBDIM0005Tag0011',
 'OBDIM0005Tag0012',
 'OBDIM0005Tag0013',
 'OBDIM0005Tag0015',
 'OBDIM0005Tag0016',
 'OBDIM0005Tag0017',
 'OBDIM0005Tag0018',
 'OBDIM0005Tag0019',
 'OBDIM0005Tag0020']

In [6]:
tag_df = pd.DataFrame()

for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df

,ORDER_CODE,TAG_ID
0,000002165753,OBDIM0005EX
1,000003745904,OBDIM0005EX
2,000004598254,OBDIM0005EX
3,000006786795,OBDIM0005EX
4,000007081060,OBDIM0005EX
...,...,...
100,616574711105,OBDIM0005Tag0020
101,617372941459,OBDIM0005Tag0020
102,366403672858,OBDIM0005Tag0020
103,629053262658,OBDIM0005Tag0020


In [10]:
tag_df.ORDER_CODE.drop_duplicates()

0         000002165753
1         000003745904
2         000004598254
3         000006786795
4         000007081060
              ...     
719187    385693808715
719188    385695571308
719189    385697035476
719190    385704783135
719191    385707345400
Name: ORDER_CODE, Length: 719192, dtype: object

In [7]:
tag_df.to_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\脾脏\tagall.pkl")

#### 关联分析

In [8]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np

In [9]:
tag_df1 = tag_df[tag_df.TAG_ID != 'OBDIM0005EX']
tag_df1

,ORDER_CODE,TAG_ID
0,5kfotv2anq74tbv,OBDIM0005Tag0001
1,636071238995,OBDIM0005Tag0001
2,5khct5gh4ih6fc9,OBDIM0005Tag0001
3,636428133961,OBDIM0005Tag0001
4,002300805886,OBDIM0005Tag0001
...,...,...
100,616574711105,OBDIM0005Tag0020
101,617372941459,OBDIM0005Tag0020
102,366403672858,OBDIM0005Tag0020
103,629053262658,OBDIM0005Tag0020


In [11]:
tag_df1.ORDER_CODE.drop_duplicates()

0      5kfotv2anq74tbv
1         636071238995
2      5khct5gh4ih6fc9
3         636428133961
4         002300805886
            ...       
89     5khbnq55jrxvw7j
99        979742189528
100       616574711105
102       366403672858
103       629053262658
Name: ORDER_CODE, Length: 13593, dtype: object

In [ ]:
def PrepareData (tagall):
    ordercode_list = []
    taglist = []
    for order_code in np.unique(tagall.ORDER_CODE):
        taglist_ = tagall.loc[tagall.ORDER_CODE == order_code, 'TagName'].values.tolist() 
        ordercode_list.append(order_code)
        taglist.append(taglist_)
    df = pd.DataFrame({"order_code": ordercode_list, "taglist":taglist})
    return df

In [ ]:
def AprioriAnalysis(dfData,min_supp,min_conf):
    DataList = dfData.taglist.values.tolist()
    te = TransactionEncoder()
    df_tf = te.fit_transform(DataList)
    df = pd.DataFrame(df_tf,columns=te.columns_)

    frequent_itemsets = apriori(df,min_support= min_supp,use_colnames= True)
    rules = association_rules(frequent_itemsets,metric = 'confidence',min_threshold = min_conf)
    # rules = rules.drop(rules[rules.lift <1.0].index)
    rules.rename(columns = {'antecedents':'from','consequents':'to','support':'sup','confidence':'conf'},inplace = True)
    rules.sort_values(by = ['sup','conf'],ascending = (False,False))
    return rules